In [174]:
import pandas as pd
from sklearn import preprocessing
from datetime import datetime
from sklearn.svm import SVR
import numpy as np

In [191]:
df = pd.read_csv("Data/ticket.csv", delimiter="\t")

#### Removing redundant sections and rows with NaN for CheckinTime

In [192]:
df.drop("TIME SECTION",1, inplace = True)
df.drop("TICKET",1, inplace = True)
df.drop("TICKET NUM",1, inplace = True)
df.drop("TABLE TYPE",1, inplace = True)
df.drop("STATUS",1, inplace = True)
df.drop("CHANNEL",1, inplace = True)
df = df.dropna()

In [196]:
df.head(5)

,REST_ID,TABLE SIZE,TIME (SEC),MONTH,DAY,WAIT TIME
0,113442,5,40159,1,5,442
1,113442,4,40545,1,5,55
2,113442,2,40631,1,5,10
3,113442,2,41681,1,5,12
4,113442,5,42660,1,5,576


#### Parsing the DateTime and adding columns based on it

In [193]:
dater = lambda c: datetime.strptime(c, "%Y-%m-%d %H:%M:%S.%f")
dater2 = lambda c: datetime.strptime(c, "%Y-%m-%d %H:%M:%S")

In [194]:
df["TIME CHECKIN"] = df["CHECKINTIME"].apply(lambda a: dater2(a).hour*3600 + dater2(a).minute*60 + dater2(a).second)
df["TIME (SEC)"] = df["CREATE TIME"].apply(lambda a: dater(a).hour*3600 + dater(a).minute*60 + dater(a).second)
df["MONTH"] = df["CREATE TIME"].apply(lambda a: int(a.split()[0].split('-')[1]))
df["DAY"] = df["CREATE TIME"].apply(lambda a: dater(a).isoweekday())
df["WAIT TIME"] = df["TIME CHECKIN"] - df["TIME (SEC)"]

In [195]:
df.drop("CREATE TIME",1, inplace = True)
df.drop("CHECKINTIME",1, inplace = True)
df.drop("TIME CHECKIN",1, inplace = True)

#### Setting up the regression model and training on it

In [197]:
all_data = np.array(df)

In [208]:
X = [i[:5] for i in all_data[:-1000]]
# a = all_data[0]
# a[5]

In [209]:
Y = [i[5] for i in all_data[:-1000]]

In [ ]:
clf = SVR(kernel = "linear")
clf.fit(X,Y)

In [207]:
clf.predict([113442,5,40159,1,5])

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([ 212.19999991])

In [213]:
test_x = [i[:5] for i in all_data[-1000:]]

In [217]:
test_y = [i[5] for i in all_data[-1000:]]
clf.score(test_x, test_y)

-0.28755138180359641